![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column                   | Description                                                                      |
|------------------------- |--------------------------------------------------------------------------------- |
| `student_id`             | A unique ID for each student.                                                    |
| `city`                   | A code for the city the student lives in.                                        |
| `city_development_index` | A scaled development index for the city.                                         |
| `gender`                 | The student's gender.                                                            |
| `relevant_experience`    | An indicator of the student's work relevant experience.                          |
| `enrolled_university`    | The type of university course enrolled in (if any).                              |
| `education_level`        | The student's education level.                                                   |
| `major_discipline`       | The educational discipline of the student.                                       |
| `experience`             | The student's total work experience (in years).                                  |
| `company_size`           | The number of employees at the student's current employer.                       |
| `company_type`           | The type of company employing the student.                                       |
| `last_new_job`           | The number of years between the student's current and previous jobs.             |
| `training_hours`         | The number of hours of training completed.                                       |
| `job_change`             | An indicator of whether the student is looking for a new job (`1`) or not (`0`). |

In [73]:
# Import necessary libraries
import pandas as pd

# Load the dataset
ds_jobs = pd.read_csv("customer_train.csv")

# View the dataset
ds_jobs.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [74]:
# Perform EDA to identify ordinal, nominal, and binary categorical features
for column in df.select_dtypes("object").columns:
    print(df_transformed[column].value_counts(), '\n')

# Define mappings for ordered categorical columns
ordered_categories = {
    'enrolled_university': ['no_enrollment', 'Part time course', 'Full time course'],
    'education_level': ['Primary School', 'High School', 'Graduate', 'Masters', 'Phd'],
    'experience': ['<1'] + list(map(str, range(1, 21))) + ['>20'],
    'company_size': ['<10', '10-49', '50-99', '100-499', '500-999', '1000-4999', '5000-9999', '10000+'],
    'last_new_job': ['never', '1', '2', '3', '4', '>4']
}

# Define mappings for binary categorical columns
binary_categories = {
    'relevant_experience': {'No relevant experience': False, 'Has relevant experience': True},
    'job_change': {0.1: False, 1.0: True}
}

# Update data types based on column categories
for column in df_transformed:

    # Convert binary categorical columns to boolean
    if column in binary_categories:
        df_transformed[column] = df_transformed[column].map(binary_categories[column])

    # Convert integer columns to int32
    elif column in ['student_id', 'training_hours']:
        df_transformed[column] = df_transformed[column].astype('int32')

    # Convert float columns to float16
    elif column == 'city_development_index':
        df_transformed[column] = df_transformed[column].astype('float16')

    # Apply ordered categories to specified columns
    elif column in ordered_categories:
        cat_type = pd.CategoricalDtype(ordered_categories[column], ordered=True)
        df_transformed[column] = df_transformed[column].astype(cat_type)

    # Convert other columns to standard categories
    else:
        df_transformed[column] = df_transformed[column].astype('category')

# Filter records for individuals with 10+ years of experience at companies with 1000+ employees
df_transformed = df_transformed[(df_transformed['experience'] >= '10') & (df_transformed['company_size'] >= '1000-4999')]

city_103    805
city_16     211
city_114    167
city_21     157
city_160     72
           ... 
city_55       0
city_121      0
city_120      0
city_2        0
city_129      0
Name: city, Length: 123, dtype: int64 

Male      1654
Female     145
Other       22
Name: gender, dtype: int64 

Series([], Name: relevant_experience, dtype: int64) 

no_enrollment       1960
Full time course     122
Part time course     103
Name: enrolled_university, dtype: int64 

Graduate          1243
Masters            744
Phd                115
High School         75
Primary School       7
Name: education_level, dtype: int64 

STEM               1892
Humanities           73
Business Degree      48
Other                35
No Major             28
Arts                 21
Name: major_discipline, dtype: int64 

>20    859
10     245
15     172
11     149
14     147
16     128
12     107
13     103
17      91
19      83
18      68
20      49
2        0
4        0
5        0
1        0
3        0
9        0
8    